# Primitives examples

<details>
<summary><b>Package versions</b></summary>

The code on this page was developed using the following requirements.
We recommend using these versions or newer.

```
qiskit[all]~=2.1.1
qiskit-ibm-runtime~=0.40.1
```
</details>

The examples in this section illustrate some common ways to use primitives. Before running these examples, follow the instructions in [Install and set up.](install-qiskit)

<Admonition type="note">
    These examples all use the primitives from Qiskit Runtime, but you could use the base primitives instead.
</Admonition>

## Estimator examples

Efficiently calculate and interpret expectation values of the quantum operators required for many algorithms with Estimator. Explore uses in molecular modeling, machine learning, and complex optimization problems.

### Run a single experiment

Use Estimator to determine the expectation value of a single circuit-observable pair.

In [1]:
import numpy as np
from qiskit.circuit.library import iqp
from qiskit.transpiler import generate_preset_pass_manager
from qiskit.quantum_info import SparsePauliOp, random_hermitian
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator

n_qubits = 50

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

mat = np.real(random_hermitian(n_qubits, seed=1234))
circuit = iqp(mat)
observable = SparsePauliOp("Z" * 50)

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(circuit)
isa_observable = observable.apply_layout(isa_circuit.layout)

estimator = Estimator(mode=backend)
job = estimator.run([(isa_circuit, isa_observable)])
result = job.result()

print(f" > Expectation value: {result[0].data.evs}")
print(f" > Metadata: {result[0].metadata}")

 > Expectation value: 0.01904296875
 > Metadata: {'shots': 4096, 'target_precision': 0.015625, 'circuit_metadata': {}, 'resilience': {}, 'num_randomizations': 32}


### Run multiple experiments in a single job

Use Estimator to determine the expectation values of multiple circuit-observable pairs.

In [2]:
import numpy as np
from qiskit.circuit.library import iqp
from qiskit.transpiler import generate_preset_pass_manager
from qiskit.quantum_info import SparsePauliOp, random_hermitian
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator

n_qubits = 50

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

rng = np.random.default_rng()
mats = [np.real(random_hermitian(n_qubits, seed=rng)) for _ in range(3)]

pubs = []
circuits = [iqp(mat) for mat in mats]
observables = [
    SparsePauliOp("X" * 50),
    SparsePauliOp("Y" * 50),
    SparsePauliOp("Z" * 50),
]

# Get ISA circuits
pm = generate_preset_pass_manager(optimization_level=1, backend=backend)

for qc, obs in zip(circuits, observables):
    isa_circuit = pm.run(qc)
    isa_obs = obs.apply_layout(isa_circuit.layout)
    pubs.append((isa_circuit, isa_obs))

estimator = Estimator(backend)
job = estimator.run(pubs)
job_result = job.result()

for idx in range(len(pubs)):
    pub_result = job_result[idx]
    print(f">>> Expectation values for PUB {idx}: {pub_result.data.evs}")
    print(f">>> Standard errors for PUB {idx}: {pub_result.data.stds}")

>>> Expectation values for PUB 0: 0.4423963133640553
>>> Standard errors for PUB 0: 0.3699949141329328
>>> Expectation values for PUB 1: 0.13313161875945537
>>> Standard errors for PUB 1: 0.37723006111708396
>>> Expectation values for PUB 2: 1.0819672131147542
>>> Standard errors for PUB 2: 1.1199392940463067


### Run parameterized circuits

Use Estimator to run three experiments in a single job, leveraging parameter values to increase circuit reusability.

In [3]:
import numpy as np

from qiskit.circuit import QuantumCircuit, Parameter
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator

service = QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator=False)

# Step 1: Map classical inputs to a quantum problem
theta = Parameter("θ")

chsh_circuit = QuantumCircuit(2)
chsh_circuit.h(0)
chsh_circuit.cx(0, 1)
chsh_circuit.ry(theta, 0)

number_of_phases = 21
phases = np.linspace(0, 2 * np.pi, number_of_phases)
individual_phases = [[ph] for ph in phases]

ZZ = SparsePauliOp.from_list([("ZZ", 1)])
ZX = SparsePauliOp.from_list([("ZX", 1)])
XZ = SparsePauliOp.from_list([("XZ", 1)])
XX = SparsePauliOp.from_list([("XX", 1)])
ops = [ZZ, ZX, XZ, XX]

# Step 2: Optimize problem for quantum execution.

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
chsh_isa_circuit = pm.run(chsh_circuit)
isa_observables = [
    operator.apply_layout(chsh_isa_circuit.layout) for operator in ops
]

# Step 3: Execute using Qiskit primitives.

# Reshape observable array for broadcasting
reshaped_ops = np.fromiter(isa_observables, dtype=object)
reshaped_ops = reshaped_ops.reshape((4, 1))

estimator = Estimator(backend, options={"default_shots": int(1e4)})
job = estimator.run([(chsh_isa_circuit, reshaped_ops, individual_phases)])
# Get results for the first (and only) PUB
pub_result = job.result()[0]
print(f">>> Expectation values: {pub_result.data.evs}")
print(f">>> Standard errors: {pub_result.data.stds}")
print(f">>> Metadata: {pub_result.metadata}")

>>> Expectation values: [[ 1.12173088  1.07596426  0.90994809  0.63893791  0.3580565   0.01017036
  -0.33472449 -0.64073268 -0.90246988 -1.05831569 -1.11185965 -1.03049677
  -0.90097424 -0.67842283 -0.36044952 -0.00687995  0.32934019  0.65419345
   0.89588906  1.05053836  1.09630498]
 [ 0.01256339  0.38049111  0.67573068  0.90815332  1.04395754  1.1166457
   1.05741831  0.88302655  0.61919544  0.33083583 -0.05115093 -0.34489485
  -0.65718473 -0.91473414 -1.07357123 -1.10438144 -1.04754708 -0.8878126
  -0.64462134 -0.27579623  0.00747821]
 [-0.01675118 -0.33741665 -0.63026319 -0.86089106 -1.01344646 -1.08404072
  -1.02271943 -0.87435183 -0.6335536  -0.33861316  0.00448692  0.33532275
   0.63325447  0.86836926  1.01165169  1.06100785  1.01374559  0.85131895
   0.61799893  0.33622014  0.00927298]
 [ 1.0657939   1.02271943  0.85371198  0.62487888  0.30929859 -0.02063985
  -0.35775737 -0.60782857 -0.88183003 -1.01763425 -1.06728954 -1.02930025
  -0.86687362 -0.59406867 -0.31737506  0.017050

### Use sessions and advanced options

Explore sessions and advanced options to optimize circuit performance on QPUs.


<Admonition type="caution">
The following code block will return an error for Open Plan users because it uses sessions. Open Plan workloads can run only in [job mode](/docs/guides/execution-modes#job-mode) or [batch mode](/docs/guides/execution-modes#batch-mode).
</Admonition>

In [4]:
import numpy as np
from qiskit.circuit.library import iqp
from qiskit.transpiler import generate_preset_pass_manager
from qiskit.quantum_info import SparsePauliOp, random_hermitian
from qiskit_ibm_runtime import (
    QiskitRuntimeService,
    Session,
    EstimatorV2 as Estimator,
)

n_qubits = 50

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

rng = np.random.default_rng(1234)
mat = np.real(random_hermitian(n_qubits, seed=rng))
circuit = iqp(mat)
mat = np.real(random_hermitian(n_qubits, seed=rng))
another_circuit = iqp(mat)
observable = SparsePauliOp("X" * 50)
another_observable = SparsePauliOp("Y" * 50)

pm = generate_preset_pass_manager(optimization_level=1, backend=backend)
isa_circuit = pm.run(circuit)
another_isa_circuit = pm.run(another_circuit)
isa_observable = observable.apply_layout(isa_circuit.layout)
another_isa_observable = another_observable.apply_layout(
    another_isa_circuit.layout
)

with Session(backend=backend) as session:
    estimator = Estimator(mode=session)

    estimator.options.resilience_level = 1

    job = estimator.run([(isa_circuit, isa_observable)])
    another_job = estimator.run(
        [(another_isa_circuit, another_isa_observable)]
    )
    result = job.result()
    another_result = another_job.result()

    # first job
    print(f" > Expectation value: {result[0].data.evs}")
    print(f" > Metadata: {result[0].metadata}")

    # second job
    print(f" > Another Expectation value: {another_result[0].data.evs}")
    print(f" > More Metadata: {another_result[0].metadata}")

 > Expectation value: 0.7420289855072464
 > Metadata: {'shots': 4096, 'target_precision': 0.015625, 'circuit_metadata': {}, 'resilience': {}, 'num_randomizations': 32}
 > Another Expectation value: 0.2440087145969499
 > More Metadata: {'shots': 4096, 'target_precision': 0.015625, 'circuit_metadata': {}, 'resilience': {}, 'num_randomizations': 32}


<span id="sampler-examples"></span>
## Sampler examples

Generate entire error-mitigated quasi-probability distributions sampled from quantum circuit outputs. Leverage Sampler’s capabilities for search and classification algorithms like Grover’s and QVSM.

### Run a single experiment

Use Sampler to return the measurement outcome as bitstrings or counts of a single circuit.

In [5]:
import numpy as np
from qiskit.circuit.library import iqp
from qiskit.transpiler import generate_preset_pass_manager
from qiskit.quantum_info import random_hermitian
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler

n_qubits = 127

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

mat = np.real(random_hermitian(n_qubits, seed=1234))
circuit = iqp(mat)
circuit.measure_all()

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(circuit)

sampler = Sampler(backend)
job = sampler.run([isa_circuit])
result = job.result()

# Get results for the first (and only) PUB
pub_result = result[0]

print(f" > First ten results: {pub_result.data.meas.get_bitstrings()[:10]}")

 > First ten results: ['0100111100101110001000000000100000001010010101110000010001000010100100000000000001100110001010011011001000001100001000011001000', '0010100101000001000000011101000010010010010000101000010101101001011100001001110100101000000010010000110000111011000001011011000', '0011111101000000011010011101001001010010000111000000100010110110101110000000110100110000101010000111000000000010000000000110110', '0001100100101001011100000001001111111001010100010001001010010111100000010001101100001100000000101000001000000000000001100010000', '1100101000110101001011111110010011010011100100111010111000011001000101000000011000001110111000010000010011010001000110100011100', '1001100011110001000001000100001010001010000111011110100010110011101000000100010001100101000010000011001110010100010000000011000', '1111000010100111001100010001000000000010110001010010111001000111011111000000110100011111011000000000111010101111001000000100001', '00010011001000000001001000111111001111111001100010001001100

### Run multiple experiments in a single job

Use Sampler to return the measurement outcome as bitstrings or counts of multiple circuits in one job.

In [6]:
import numpy as np
from qiskit.circuit.library import iqp
from qiskit.transpiler import generate_preset_pass_manager
from qiskit.quantum_info import random_hermitian
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler

n_qubits = 127

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

rng = np.random.default_rng()
mats = [np.real(random_hermitian(n_qubits, seed=rng)) for _ in range(3)]
circuits = [iqp(mat) for mat in mats]
for circuit in circuits:
    circuit.measure_all()

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuits = pm.run(circuits)

sampler = Sampler(mode=backend)
job = sampler.run(isa_circuits)
result = job.result()

for idx, pub_result in enumerate(result):
    print(
        f" > First ten results for pub {idx}: {pub_result.data.meas.get_bitstrings()[:10]}"
    )

 > First ten results for pub 0: ['1111000100110000011111011000011111100001000011011010000010111000010100100011110111010010110010011000110011110000001010001010000', '0000010110110000010100010001101100110111000000001011101010100010001000000101010000011000000010001011010111101111100100001100011', '1000100110001001101110001010111000010010111100110000101000010111111011000000110110010110111001011001001010001100000100000000000', '0011110001101010000100010101011111010101011010010010010100110001111011101011000110000000001011010001110100101101000110011111100', '0101010111110000011011101010010110110101101000001100100010111101001001010110111010000000110011010100011011100100010001010100001', '1001000111010001110001111100000110011101011000111110101010010100001010101101001000000111110001000000001110001110110101000010001', '0111111100100001001000111110101100101101100111011011100111000010001000000001011011100011001100110001011001110100010100011001000', '1001100110110011000110111101101100001100010101101

### Run parameterized circuits

Run several experiments in a single job, leveraging parameter values to increase circuit reusability.

In [7]:
import numpy as np
from qiskit.circuit.library import real_amplitudes
from qiskit.transpiler import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler

n_qubits = 127

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

# Step 1: Map classical inputs to a quantum problem
circuit = real_amplitudes(num_qubits=n_qubits, reps=2)
circuit.measure_all()

# Define three sets of parameters for the circuit
rng = np.random.default_rng(1234)
parameter_values = [
    rng.uniform(-np.pi, np.pi, size=circuit.num_parameters) for _ in range(3)
]

# Step 2: Optimize problem for quantum execution.

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(circuit)

# Step 3: Execute using Qiskit primitives.
sampler = Sampler(backend)
job = sampler.run([(isa_circuit, parameter_values)])
result = job.result()
# Get results for the first (and only) PUB
pub_result = result[0]
# Get counts from the classical register "meas".
print(
    f" >> First ten results for the meas output register: {pub_result.data.meas.get_bitstrings()[:10]}"
)

 >> First ten results for the meas output register: ['1110100000000110110100001001000001100111000001110111001000100010010010110101010100010100000100110000011001001101111110001000111', '0000001100001111011000001111110010000101110000110011011111101001101001001011011111001110010101111111101100010111111010011001110', '1110100100001000011010011000000100110010000101110011001000001101010001010011011010010100000001101000110100111001110000001011010', '0000011110011111111011101010100000011000010000011100000100100011011000110111001010111101000100000100001001011101010111101101100', '0010101011001101011010100001000000001000000100011101010110100010100001101111000101010100000001010101010011111100001100110000001', '1000100001001001011111100100010011111010110001001010100110000101101011001011111110101110110000000001111001100010111001010000101', '0100111000001101110110001101010001001111010101101111000000101010100000100011000100011011110001101011111101111110100110010110111', '01101001000010000101000111110

### Use sessions and advanced options

Explore sessions and advanced options to optimize circuit performance on QPUs.


<Admonition type="caution">
The following code block will return an error for users on the Open Plan, because it uses sessions. Workloads on the Open Plan can run only in [job mode](/docs/guides/execution-modes#job-mode) or [batch mode](/docs/guides/execution-modes#batch-mode).
</Admonition>

In [8]:
import numpy as np
from qiskit.circuit.library import iqp
from qiskit.quantum_info import random_hermitian
from qiskit.transpiler import generate_preset_pass_manager
from qiskit_ibm_runtime import Session, SamplerV2 as Sampler
from qiskit_ibm_runtime import QiskitRuntimeService

n_qubits = 127

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

rng = np.random.default_rng(1234)
mat = np.real(random_hermitian(n_qubits, seed=rng))
circuit = iqp(mat)
circuit.measure_all()
mat = np.real(random_hermitian(n_qubits, seed=rng))
another_circuit = iqp(mat)
another_circuit.measure_all()

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(circuit)
another_isa_circuit = pm.run(another_circuit)

with Session(backend=backend) as session:
    sampler = Sampler(mode=session)
    job = sampler.run([isa_circuit])
    another_job = sampler.run([another_isa_circuit])
    result = job.result()
    another_result = another_job.result()

# first job

print(
    f" > The first ten measurement results of job 1: {result[0].data.meas.get_bitstrings()[:10]}"
)

 > The first ten measurement results of job 1: ['0100010110001110000000101000010010100010101000000100110111100100110110000001100001000010011110000000001000000011011010000100100', '1111000010101001010010111110000101101010001010010000000111010100001100010000000110101000101001100011110000001101000001000100001', '0010011000110010000000000010000011101001100110000110101111101010000101101110100100000001100100100000010000001001000000001100100', '0100011011010100000111111101011000001110010101000010100101110110101100001010110010000000000000100100110011001011000001000100001', '0110110110000100010001101010010010101111000101110000010001110110010000000001010110100001000100011011101100001010001000000010000', '0100101001100010001010100000011000000110110001010101000110011000111001000010100010001110101001001001110000010000000000000000100', '0111101000110101000000000011100010100010011001001010010010000001010100100111010010000101100101101000010000001101000110010001011', '0101100100000011001110001100000000

In [9]:
# second job
print(
    " > The first ten measurement results of job 2:",
    another_result[0].data.meas.get_bitstrings()[:10],
)

 > The first ten measurement results of job 2: ['0100011001111101001110010100111100010111010010001100110000111110000001100010101111011011001000000101000000111100101000010110000', '1101100101010100000000000100010010000000001011010010110001010100010101000010000000111101010010011110011000000100010010000000000', '1110111000001100000001010001010001000111010011100001111100110000101010110111010010011111011000011001100000100010010000010000000', '0011101111110000100100000000111011001000101100001010110001010100101001000010100000001010100001000110100000001101000100010000010', '0100111001111000100111100101110001000100011010101100100000100100010011000110100010011100001010001000010000001100001010010010100', '0000001101101111110110011001101110110001000001001100010000110110000011000100101110100101011011011010101000001000100000010010001', '0001000001000001011010011101101001000010100011000010010101001100110001010100000001100010000011010000011000000011010000010110000', '1100010011100111101100101110000011

## Next steps

<Admonition type="tip" title="Recommendations">
    - [Specify advanced runtime options.](runtime-options-overview)
    - Practice with primitives by working through the [Cost function lesson](/learning/courses/variational-algorithm-design/cost-functions) in IBM Quantum Learning.
    - Learn how to transpile locally in the [Transpile](./transpile/) section.
    - Try the [Compare transpiler settings](/docs/tutorials/circuit-transpilation-settings) tutorial.
    - Read [Migrate to V2 primitives](/docs/migration-guides/v2-primitives).
    - Understand the [Job limits](/docs/guides/job-limits) when sending a job to an IBM&reg; QPU.
</Admonition>